## Sentiment Analysis using BERT pre-trained model

Here BERT is used from the hugging face tranformers library. Bert 
architecture can be defined or changed using transformers.BertConfig. If we use the default values while instantiating BERT, the base model discussed in [this paper](https://arxiv.org/pdf/1810.04805.pdf). This is a masked language modelling focused uncased model. 

Uncased model indicates that BERT doesn't differentiate between different cases. It is useful here, because we are working SMILE data from twitter which contains user tweets. 

### Exploratory data Analysis

In [1]:
import pandas as pd

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
# reading and viewing the dataset
# because the dataset is just a collection of id, tweets and emotions with no column names,  
# we add names while reading in the data
df = pd.read_csv("/content/drive/My Drive/Colab Notebooks/Data/smile-annotations-final.csv", names = ['id', 'tweets', 'emotions'])

In [4]:
df.set_index('id', inplace=True)

In [5]:
df.head()

,tweets,emotions
id,,
611857364396965889,@aandraous @britishmuseum @AndrewsAntonio Merc...,nocode
614484565059596288,Dorian Gray with Rainbow Scarf #LoveWins (from...,happy
614746522043973632,@SelectShowcase @Tate_StIves ... Replace with ...,happy
614877582664835073,@Sofabsports thank you for following me back. ...,happy
611932373039644672,@britishmuseum @TudorHistory What a beautiful ...,happy


In [6]:
# to view all the different emotion categories, we do:

df.emotions.value_counts()

nocode               1572
happy                1137
not-relevant          214
angry                  57
surprise               35
sad                    32
happy|surprise         11
happy|sad               9
disgust|angry           7
disgust                 6
sad|angry               2
sad|disgust             2
sad|disgust|angry       1
Name: emotions, dtype: int64

In [7]:
# to simplify the process, removing multilabel classification problem

df = df[~df.emotions.str.contains('\|')]

In [8]:
# can also removing nocode, because that doesn't add anything to the model:
df = df[df.emotions != 'nocode']

In [9]:
df.emotions.value_counts()

happy           1137
not-relevant     214
angry             57
surprise          35
sad               32
disgust            6
Name: emotions, dtype: int64

In [10]:
possible_emotions = df.emotions.unique()

In [11]:
# encoding the emotions
dict_emotions = {}
for index, emotion in enumerate(possible_emotions):
    dict_emotions[emotion] = index

In [12]:
df['emotion_label'] = df.emotions.replace(dict_emotions)

### Creating Training Data

In [13]:
from sklearn.model_selection import train_test_split

In [14]:
# because test data needs to see all examples of training data and 
# there is imbalanced class problem, using stratified splitting over different emotions

X_train, X_val, y_train, y_val = train_test_split(df.index.values, 
                                                  df.emotion_label.values, 
                                                  test_size=0.15, 
                                                  random_state=17, 
                                                  stratify=df.emotion_label.values)

In [15]:
df['data_type'] = ['not_set']*df.shape[0]

In [16]:
df.loc[X_train, 'data_type'] = 'train'
df.loc[X_val, 'data_type'] = 'val'

In [17]:
df.groupby(['emotions', 'emotion_label', 'data_type']).count()

tweets
emotions     emotion_label data_type        
angry        2             train          48
                           val             9
disgust      3             train           5
                           val             1
happy        0             train         966
                           val           171
not-relevant 1             train         182
                           val            32
sad          4             train          27
                           val             5
surprise     5             train          30
                           val             5

### Feature Engineering - Tokenizing and Encoding the Data

In [18]:
!pip install -U pip
!pip --version
!pip install transformers


Requirement already up-to-date: pip in /usr/local/lib/python3.6/dist-packages (20.2.3)
pip 20.2.3 from /usr/local/lib/python3.6/dist-packages/pip (python 3.6)


In [19]:
from transformers import BertTokenizer
from torch.utils.data import TensorDataset
import torch

In [20]:
# we use the bert-base-uncased pretrained model.

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', 
                                          do_lower_case=True)

In [21]:
encoded_data_train = tokenizer.batch_encode_plus(
    df[df.data_type=='train'].tweets.values, 
    add_special_tokens=True, # required for signalling end of line.
    return_attention_mask=True, # required for using attention mechanism
    pad_to_max_length=True, # using a fixed max length
    max_length=256, 
    return_tensors='pt'
)

encoded_data_val = tokenizer.batch_encode_plus(
    df[df.data_type=='val'].tweets.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=256, 
    return_tensors='pt'
)


input_ids_train = encoded_data_train['input_ids']
attention_masks_train = encoded_data_train['attention_mask']
labels_train = torch.tensor(df[df.data_type=='train'].emotion_label.values)

input_ids_val = encoded_data_val['input_ids']
attention_masks_val = encoded_data_val['attention_mask']
labels_val = torch.tensor(df[df.data_type=='val'].emotion_label.values)

Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:1773: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
Truncation was not explicitely activated but `max_length` is provided a specific value, please use

In [22]:
dataset_train = TensorDataset(input_ids_train, attention_masks_train, labels_train)
dataset_val = TensorDataset(input_ids_val, attention_masks_val, labels_val)

In [23]:
len(dataset_train)

1258

In [24]:
len(dataset_val)

223

### Setting up pre-trained model for training

In [25]:
from transformers import BertForSequenceClassification

In [26]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased",
                                                      num_labels=len(dict_emotions),
                                                      output_attentions=False,
                                                      output_hidden_states=False)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

### Creating data loaders

In [27]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

In [28]:
batch_size = 32

dataloader_train = DataLoader(dataset_train, 
                              sampler=RandomSampler(dataset_train), 
                              batch_size=batch_size)

dataloader_validation = DataLoader(dataset_val, 
                                   sampler=SequentialSampler(dataset_val), 
                                   batch_size=batch_size)

### Setting up optimizer and scheduler

In [29]:
from transformers import AdamW, get_linear_schedule_with_warmup

In [30]:
optimizer = AdamW(model.parameters(),
                  lr=1e-5, 
                  eps=1e-8)

In [31]:
epochs = 10

scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps=0,
                                            num_training_steps=len(dataloader_train)*epochs)

### Defining Performance metrics

In [32]:
import numpy as np
from sklearn.metrics import f1_score

In [33]:
def f1_score_func(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat, preds_flat, average='weighted')

In [34]:
def accuracy_per_class(preds, labels):
    label_dict_inverse = {v: k for k, v in dict_emotions.items()}
    
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    for label in np.unique(labels_flat):
        y_preds = preds_flat[labels_flat==label]
        y_true = labels_flat[labels_flat==label]
        print(f'Class: {label_dict_inverse[label]}')
        print(f'Accuracy: {len(y_preds[y_preds==label])}/{len(y_true)}\n')

### Creating Training Loop

In [35]:
import random
import numpy as np

seed_val = 17
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

In [36]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

print(device)

cuda


In [37]:
def evaluate(dataloader_val):

    model.eval()
    
    loss_val_total = 0
    predictions, true_vals = [], []
    
    for batch in dataloader_val:
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }

        with torch.no_grad():        
            outputs = model(**inputs)
            
        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)
    
    loss_val_avg = loss_val_total/len(dataloader_val) 
    
    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)
            
    return loss_val_avg, predictions, true_vals

In [38]:
from tqdm.notebook import tqdm

In [39]:
for epoch in tqdm(range(1, epochs+1)):
    
    model.train()
    
    loss_train_total = 0

    progress_bar = tqdm(dataloader_train, desc='Epoch {:1d}'.format(epoch), leave=False, disable=False)
    for batch in progress_bar:

        model.zero_grad()
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }       

        outputs = model(**inputs)
        
        loss = outputs[0]
        loss_train_total += loss.item()
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()
        scheduler.step()
        
        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})
         
        
    torch.save(model.state_dict(), f'finetuned_BERT_epoch_{epoch}.model')
        
    tqdm.write(f'\nEpoch {epoch}')
    
    loss_train_avg = loss_train_total/len(dataloader_train)            
    tqdm.write(f'Training loss: {loss_train_avg}')
    
    val_loss, predictions, true_vals = evaluate(dataloader_validation)
    val_f1 = f1_score_func(predictions, true_vals)
    tqdm.write(f'Validation loss: {val_loss}')
    tqdm.write(f'F1 Score (Weighted): {val_f1}')


Epoch 1
Training loss: 1.012143647670746
Validation loss: 0.7918027469090053
F1 Score (Weighted): 0.7043271626231267



Epoch 2
Training loss: 0.7032025441527366
Validation loss: 0.6862178359712873
F1 Score (Weighted): 0.7225639395972363



Epoch 3
Training loss: 0.5556432992219925
Validation loss: 0.6250344003949847
F1 Score (Weighted): 0.7515202672997452



Epoch 4
Training loss: 0.4661639902740717
Validation loss: 0.5643291175365448
F1 Score (Weighted): 0.7836900406487414



Epoch 5
Training loss: 0.390165601298213
Validation loss: 0.5441189621176038
F1 Score (Weighted): 0.7828110977336797



Epoch 6
Training loss: 0.3454963952302933
Validation loss: 0.5303412803581783
F1 Score (Weighted): 0.7867933922064038



Epoch 7
Training loss: 0.30545947290956976
Validation loss: 0.5375396055834634
F1 Score (Weighted): 0.7902172903312981



Epoch 8
Training loss: 0.2828568108379841
Validation loss: 0.5390337790761676
F1 Score (Weighted): 0.7867933922064038



Epoch 9
Training loss: 0.2695657029747963
Validation loss: 0.5524621307849884
F1 Score (Weighted): 0.7988139625419411



Epoch 10
Training loss: 0.2578167231753469
Validation loss: 0.5400097072124481
F1 Score (Weighted): 0.7953593019333184



Following the training and validation loss above - the model needs to be fine-tuned, and trained more.

In [2]:
# to run a pretrained model
# model = BertForSequenceClassification.from_pretrained("bert-base-uncased",
#                                                       num_labels=len(dict_emotions),
#                                                       output_attentions=False,
#                                                       output_hidden_states=False)

# model.to(device)
# model.load_state_dict(torch.load('Models/<<INSERT MODEL NAME HERE>>.model'), map_location=torch.device(str(device)))

In [42]:
_, predictions, true_vals = evaluate(dataloader_validation)

In [43]:
accuracy_per_class(predictions, true_vals)

Class: happy
Accuracy: 163/171

Class: not-relevant
Accuracy: 20/32

Class: angry
Accuracy: 0/9

Class: disgust
Accuracy: 0/1

Class: sad
Accuracy: 0/5

Class: surprise
Accuracy: 1/5



### Check GPU type

In [45]:
# memory footprint support libraries/code
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize

  Created wheel for gputil: filename=GPUtil-1.4.0-py3-none-any.whl size=7409 sha256=b829dea83c743d459156f074ed7e4ebab223536be4778fc01f8e276186c19f62
  Stored in directory: /root/.cache/pip/wheels/79/c1/b2/b6fc2647f693a084da25e1d31328ab3dbb565cc58fea37e973
Successfully built gputil


In [46]:

import psutil
import humanize
import os
import GPUtil as GPU
GPUs = GPU.getGPUs()

# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
def printm():
  process = psutil.Process(os.getpid())
  print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
  print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))

printm()

Gen RAM Free: 10.5 GB  | Proc size: 3.7 GB
GPU RAM Free: 4895MB | Used: 11385MB | Util  70% | Total 16280MB


In [47]:
!nvidia-smi --query-gpu=gpu_name,driver_version,memory.total --format=csv


name, driver_version, memory.total [MiB]
Tesla P100-PCIE-16GB, 418.67, 16280 MiB
